In [7]:
# Load in the relevant Python modules
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

from datetime import datetime
from datetime import timedelta

# Using plotly + cufflinks in offline mode
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import cufflinks
cufflinks.go_offline(connected=True)

import warnings
from collections import Counter
from sklearn.feature_selection import mutual_info_classif
warnings.filterwarnings('ignore')

In [8]:
# physical_details_df = pd.read_csv("physical_details.csv") 
physical_details_df = pd.read_csv("./HospitalData/September2007/physical_details.csv") 

# Preview the first 5 lines of the loaded data
physical_details_df.head()

,patient_id,medical_id,gender,entry_group,department,entry_date,exit_date,outcome,entry_day,entry_time,exit_day,exit_time,seg_type,duration,duration_hours,duration_days,age_years,age_months,physical_department
0,10000030,20151960,2,1,2,9/19/2007,9/19/2007,8,13775,1190208270,13775,1190245800,2,37530,10,0,46,5,2
1,10000067,20008256,2,1,1,9/4/2007,9/4/2007,2,13760,1188937712,13760,1188947873,1,10161,2,0,78,0,1
2,10000067,20008256,2,2,37,9/4/2007,9/6/2007,7,13760,1188947873,13762,1189090892,3,143019,39,1,78,0,37
3,10000085,20006936,2,1,1,9/4/2007,9/4/2007,7,13760,1188903838,13760,1188913393,2,9555,2,0,69,7,1
4,10000101,20186917,2,1,11,9/21/2007,9/21/2007,2,13777,1190416120,13777,1190416919,1,799,0,0,38,4,11


In [9]:
# Taking a look at how many rows and columns the train dataset contains
rows = physical_details_df.shape[0]
columns = physical_details_df.shape[1]
print("The dataset contains {0} rows and {1} columns".format(rows, columns))

The dataset contains 16460 rows and 19 columns


In [10]:
physical_details_df.isnull().values.any()

False

In [11]:
physical_details_df = physical_details_df.loc[physical_details_df['exit_time'] != 0]
# physical_details_df.duration_hours.iplot(kind='bar', title="Duration [hours] variable distribution")

### Patients currently in ED

In [12]:
requested_datetime_str = '09/14/07 19:00:00'
requested_datetime_object = datetime.strptime(requested_datetime_str, '%m/%d/%y %H:%M:%S')
print("requested_datetime_object: ", requested_datetime_object)
requested_timestamp = datetime.timestamp(requested_datetime_object)
print("requested_timestamp: ", requested_timestamp)
# initial_timestamp_object = datetime.fromtimestamp(requested_timestamp)
# print("initial_timestamp_object =", initial_timestamp_object)


# Initial count
er_mask = physical_details_df['entry_group'] == 1
entry_time_mask = physical_details_df['entry_time'] <= requested_timestamp
exit_time_mask = (physical_details_df['exit_time'] > requested_timestamp) | (physical_details_df['exit_time'] == 0) 

current_patients_df = physical_details_df.loc[er_mask & entry_time_mask & exit_time_mask]
print("The dataset contains {0} patients - initial time {1}".format(current_patients_df.shape[0], requested_datetime_object))

requested_datetime_object:  2007-09-14 19:00:00
requested_timestamp:  1189785600.0
The dataset contains 32 patients - initial time 2007-09-14 19:00:00


In [13]:
current_patients_subset_df = current_patients_df[['patient_id', 'medical_id', 'gender', 'entry_time', 'duration', 'outcome']]
current_patients_subset_df['duration'] = current_patients_subset_df.apply(lambda row: timedelta(seconds=requested_timestamp-row.entry_time), axis = 1)
current_patients_subset_df['entry_time'] = current_patients_subset_df.apply(lambda row: datetime.fromtimestamp(row.entry_time), axis = 1)
current_patients_subset_df['gender'] = current_patients_subset_df.apply(lambda row: 'Male' if (row['gender'] == 1) else 'Female', axis = 1)
current_patients_subset_df['outcome'] = current_patients_subset_df.apply(lambda row: 'Medium' if (row['outcome'] == 2) else 'Low', axis = 1)

current_patients_subset_df.sort_values(by ='entry_time', inplace=True)

current_patients_subset_df = current_patients_subset_df.rename(columns={'patient_id': 'Patient ID', 'gender': 'Gender', 'entry_time': 'Entered', 'duration': 'Wait [hours]', 'outcome': 'Severity'})
current_patients_subset_df.head()

,Patient ID,medical_id,Gender,Entered,Wait [hours],Severity
2059,10021671,20029822,Female,2007-09-14 03:17:01,15:42:59,Medium
13510,10189028,20345436,Male,2007-09-14 05:21:26,13:38:34,Low
9054,10124875,20345420,Male,2007-09-14 09:10:21,09:49:39,Low
9179,10127044,20233732,Male,2007-09-14 09:53:34,09:06:26,Low
5275,10056957,20202024,Female,2007-09-14 11:37:35,07:22:25,Low


### Calculate number of patients in ED in last 12 hours

In [14]:
requested_datetime_str = '09/14/07 19:00:00'
requested_datetime_object = datetime.strptime(requested_datetime_str, '%m/%d/%y %H:%M:%S')
print("requested_datetime_object: ", requested_datetime_object)
requested_timestamp = datetime.timestamp(requested_datetime_object)
initial_timestamp = requested_timestamp - 12*60*60
initial_timestamp_object = datetime.fromtimestamp(initial_timestamp)
print("initial_timestamp_object =", initial_timestamp_object)

# Initial count
er_mask = physical_details_df['entry_group'] == 1
initial_mask_entries = physical_details_df['entry_time'] <= initial_timestamp
initial_mask_exits = (physical_details_df['exit_time'] > initial_timestamp) | (physical_details_df['exit_time'] == 0) 
initial_physical_details_valid_df = physical_details_df.loc[er_mask & initial_mask_entries & initial_mask_exits]
print("The dataset contains {0} patients - initial time {1}".format(initial_physical_details_valid_df.shape[0], initial_timestamp_object))

requested_datetime_object:  2007-09-14 19:00:00
initial_timestamp_object = 2007-09-14 07:00:00
The dataset contains 27 patients - initial time 2007-09-14 07:00:00


In [15]:
er_mask = physical_details_df['entry_group'] == 1
entries_between_timestamps_mask = (physical_details_df['entry_time'] > initial_timestamp) & (physical_details_df['entry_time'] <= requested_timestamp)
physical_details_valid_entries_df = physical_details_df.loc[er_mask & entries_between_timestamps_mask]

entries_subset_df = physical_details_valid_entries_df[['patient_id', 'gender', 'entry_time']]
entries_subset_df['count_as'] = 1
entries_subset_df['time'] = entries_subset_df.apply(lambda row: row.entry_time, axis = 1)
entries_subset_df['date'] = entries_subset_df.apply(lambda row: datetime.fromtimestamp(row.entry_time), axis = 1)
entries_subset_df.drop(columns=['entry_time'], inplace = True)
print("The dataset contains {0} entries between {1} and {2}".format(entries_subset_df.shape[0], initial_timestamp_object, requested_datetime_object))

entries_subset_df.head()

The dataset contains 133 entries between 2007-09-14 07:00:00 and 2007-09-14 19:00:00


,patient_id,gender,count_as,time,date
38,10000386,2,1,1189769599,2007-09-14 14:33:19
39,10000396,2,1,1189767898,2007-09-14 14:04:58
256,10002702,2,1,1189769281,2007-09-14 14:28:01
294,10003436,2,1,1189746239,2007-09-14 08:03:59
544,10005783,2,1,1189769634,2007-09-14 14:33:54


In [16]:
er_mask = physical_details_df['entry_group'] == 1
exits_between_timestamps_mask = (physical_details_df['exit_time'] > initial_timestamp) & (physical_details_df['exit_time'] <= requested_timestamp)
physical_details_valid_exits_df = physical_details_df.loc[er_mask & exits_between_timestamps_mask]

exits_subset_df = physical_details_valid_exits_df[['patient_id', 'gender', 'exit_time']]
exits_subset_df['count_as'] = -1
exits_subset_df['time'] = exits_subset_df.apply(lambda row: row.exit_time, axis = 1)
exits_subset_df['date'] = exits_subset_df.apply(lambda row: datetime.fromtimestamp(row.exit_time), axis = 1)
exits_subset_df.drop(columns=['exit_time'], inplace = True)
print("The dataset contains {0} exits between {1} and {2}".format(exits_subset_df.shape[0], initial_timestamp_object, requested_datetime_object))
exits_subset_df.head()

The dataset contains 128 exits between 2007-09-14 07:00:00 and 2007-09-14 19:00:00


,patient_id,gender,count_as,time,date
38,10000386,2,-1,1189778700,2007-09-14 17:05:00
39,10000396,2,-1,1189780413,2007-09-14 17:33:33
256,10002702,2,-1,1189780761,2007-09-14 17:39:21
294,10003436,2,-1,1189767600,2007-09-14 14:00:00
544,10005783,2,-1,1189780001,2007-09-14 17:26:41


In [17]:
frames = [entries_subset_df, exits_subset_df]

count_patients_df = pd.concat(frames)
count_patients_df.sort_values(by = 'time', inplace = True)
count_patients_df.head()

,patient_id,gender,count_as,time,date
14670,10202321,1,1,1189742641,2007-09-14 07:04:01
13229,10184255,1,-1,1189742818,2007-09-14 07:06:58
6282,10071708,2,1,1189743505,2007-09-14 07:18:25
4494,10050517,2,1,1189743778,2007-09-14 07:22:58
4494,10050517,2,-1,1189743800,2007-09-14 07:23:20


In [18]:
count = initial_physical_details_valid_df.shape[0]
# count = 0
count_series = []
print('initial_count: ', count)

for index, row in count_patients_df.iterrows():
    count = count + row.count_as
    count_series.append(count)

count_patients_df['count'] = count_series
count_patients_df.head(10)

initial_count:  27


,patient_id,gender,count_as,time,date,count
14670,10202321,1,1,1189742641,2007-09-14 07:04:01,28
13229,10184255,1,-1,1189742818,2007-09-14 07:06:58,27
6282,10071708,2,1,1189743505,2007-09-14 07:18:25,28
4494,10050517,2,1,1189743778,2007-09-14 07:22:58,29
4494,10050517,2,-1,1189743800,2007-09-14 07:23:20,28
14928,10224803,1,-1,1189744244,2007-09-14 07:30:44,27
11407,10144571,1,1,1189744329,2007-09-14 07:32:09,28
4050,10044507,2,-1,1189744407,2007-09-14 07:33:27,27
8834,10121722,1,-1,1189744800,2007-09-14 07:40:00,26
10104,10134704,1,-1,1189744921,2007-09-14 07:42:01,25


In [19]:
data = [
    go.Scatter(
        x=count_patients_df['date'],
        y=count_patients_df['count']
    )
]
layout = go.Layout(
    showlegend=False,
    yaxis=dict(
        rangemode='tozero',
        autorange=True
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='axes-range-mode')

### Calculate wait times in ED (avg, max, min)

In [25]:
requested_datetime_str = '09/14/07 19:00:00'
requested_datetime_object = datetime.strptime(requested_datetime_str, '%m/%d/%y %H:%M:%S')
print("requested_datetime_object: ", requested_datetime_object)
requested_timestamp = datetime.timestamp(requested_datetime_object)

# Initial count
er_mask = physical_details_df['entry_group'] == 1
entry_time_mask = physical_details_df['entry_time'] <= requested_timestamp
exit_time_mask = (physical_details_df['exit_time'] > requested_timestamp) | (physical_details_df['exit_time'] == 0) 

valid_patients_df = physical_details_df.loc[er_mask & entry_time_mask & exit_time_mask]
print("The dataset contains {0} patients - initial time {1}".format(valid_patients_df.shape[0], requested_datetime_object))

requested_datetime_object:  2007-09-14 19:00:00
The dataset contains 32 patients - initial time 2007-09-14 19:00:00


In [26]:
avg_wait_arr = []
max_wait = 0

for index, row in valid_patients_df.iterrows():
    wait_time = (row.exit_time - row.entry_time) if (row.exit_time < requested_timestamp) else (requested_timestamp - row.entry_time)
    avg_wait_arr.append(wait_time)
    if wait_time > max_wait:
        max_wait = wait_time

avg_wait = sum(avg_wait_arr) / valid_patients_df.shape[0]

print("max_wait ", max_wait/60/60)
print("avg_wait", avg_wait/60/60)

max_wait  15.71638888888889
avg_wait 3.517248263888889


### Ward admission distribution in last 12 hours 

In [27]:
requested_datetime_str = '09/14/07 19:00:00'
requested_datetime_object = datetime.strptime(requested_datetime_str, '%m/%d/%y %H:%M:%S')
print("requested_datetime_object: ", requested_datetime_object)
requested_timestamp = datetime.timestamp(requested_datetime_object)
initial_timestamp = requested_timestamp - 12*60*60
initial_timestamp_object = datetime.fromtimestamp(initial_timestamp)
print("initial_timestamp_object =", initial_timestamp_object)

er_mask = physical_details_df['entry_group'] == 1
exits_between_timestamps_mask = (physical_details_df['exit_time'] > initial_timestamp) & (physical_details_df['exit_time'] <= requested_timestamp)
physical_details_valid_exits_df = physical_details_df.loc[er_mask & exits_between_timestamps_mask]

print("The dataset contains {0} exits between {1} and {2}".format(physical_details_valid_exits_df.shape[0], initial_timestamp_object, requested_datetime_object))
physical_details_valid_exits_df.head()

requested_datetime_object:  2007-09-14 19:00:00
initial_timestamp_object = 2007-09-14 07:00:00
The dataset contains 128 exits between 2007-09-14 07:00:00 and 2007-09-14 19:00:00


,patient_id,medical_id,gender,entry_group,department,entry_date,exit_date,outcome,entry_day,entry_time,exit_day,exit_time,seg_type,duration,duration_hours,duration_days,age_years,age_months,physical_department
38,10000386,20215610,2,1,8,9/14/2007,9/14/2007,7,13770,1189769599,13770,1189778700,2,9101,2,0,43,0,8
39,10000396,20022102,2,1,5,9/14/2007,9/14/2007,7,13770,1189767898,13770,1189780413,2,12515,3,0,56,10,5
256,10002702,20094581,2,1,1,9/14/2007,9/14/2007,2,13770,1189769281,13770,1189780761,1,11480,3,0,75,4,1
294,10003436,20051865,2,1,1,9/14/2007,9/14/2007,7,13770,1189746239,13770,1189767600,2,21361,5,0,53,9,1
544,10005783,20006981,2,1,1,9/14/2007,9/14/2007,7,13770,1189769634,13770,1189780001,2,10367,2,0,47,5,1


In [57]:
def department_label(index):
    if index == 1:
        return 'Emergency Internal Medicine Unit'
    elif index == 2:
        return 'Emergency Surgery Unit'
    elif index == 9:
        return 'Pediatric Emergency Unit'
    elif index == 11:
        return 'Emergency Maternity Unit'
    elif index == 6:
        return 'Emergency Ophthalmology Unit'
    elif index == 8:
        return 'Emergency Gynecology Unit'
    elif index == 7:
        return 'Emergency Psychiatry Unit'
    elif index == 5:
        return 'Emergency Otorhinolaryngology Unit'
    else:
        return 'Emergency Internal Medicine Unit'

In [61]:
colors = ['#eba796', '#96ebda']

fig = {'data' : [{'type' : 'pie',
                  'name' : "Patients with Trouble Sleep: Pie chart",
                 'labels' : physical_details_valid_exits_df['department'].value_counts().index.map(lambda x : department_label(x)),
                 'values' : physical_details_valid_exits_df['department'].value_counts().values,
                 'direction' : 'clockwise',
                'textinfo': 'value',
                 'marker' : {'colors' : ['#9cc359', '#e96b5c']}}], 'layout' : {'title' : 'Ward distribution in last 12 hours'}}

py.iplot(fig)

### Xrays

In [20]:
xrays_visits_df = pd.read_csv("./HospitalData/September2007/xrays_visits.csv") 
# Preview the first 5 lines of the loaded data
xrays_visits_df['entry_date'] = xrays_visits_df.apply(lambda row: datetime.fromtimestamp(row.entry_time), axis = 1)
xrays_visits_df['exit_date'] = xrays_visits_df.apply(lambda row: datetime.fromtimestamp(row.exit_time), axis = 1)

xrays_visits_df.sort_values(by ='entry_time', inplace=True)

xrays_visits_df.head()

,patient_id,medical_id,gender,department,entry_date,exit_date,outcome,entry_day,entry_time,exit_day,exit_time,duration,age_years,age_months
490,10096911,20524287,2,180,2007-09-02 10:35:00,2007-09-02 10:51:00,13,13758,1188718500,13758,1188719460,960,58,1
1524,10258538,20500183,2,180,2007-09-02 10:49:00,2007-09-02 10:55:00,13,13758,1188719340,13758,1188719700,360,52,7
1542,10259050,20501223,2,180,2007-09-02 10:51:00,2007-09-02 11:01:00,13,13758,1188719460,13758,1188720060,600,45,10
2258,10291397,20601013,1,180,2007-09-02 10:54:00,2007-09-02 12:34:00,13,13758,1188719640,13758,1188725640,6000,56,8
684,10133282,20553346,1,133,2007-09-02 10:55:00,2007-09-02 11:29:00,13,13758,1188719700,13758,1188721740,2040,62,8


In [22]:
# requested_datetime_str = '09/14/07 19:00:00'
requested_datetime_str = '09/02/07 12:00:00'

requested_datetime_object = datetime.strptime(requested_datetime_str, '%m/%d/%y %H:%M:%S')
print("requested_datetime_object: ", requested_datetime_object)
requested_timestamp = datetime.timestamp(requested_datetime_object)

entry_mask = xrays_visits_df['entry_time'] <= requested_timestamp
exit_mask = (xrays_visits_df['exit_time'] > requested_timestamp) | (xrays_visits_df['exit_time'] == 0) 
xrays_visits_in_frame_df = xrays_visits_df.loc[entry_mask & exit_mask]
print("The dataset contains {0} patients - initial time {1}".format(xrays_visits_in_frame_df.shape[0], requested_datetime_object))
xrays_visits_in_frame_df

requested_datetime_object:  2007-09-02 12:00:00
The dataset contains 14 patients - initial time 2007-09-02 12:00:00


,patient_id,medical_id,gender,department,entry_date,exit_date,outcome,entry_day,entry_time,exit_day,exit_time,duration,age_years,age_months
2258,10291397,20601013,1,180,2007-09-02 10:54:00,2007-09-02 12:34:00,13,13758,1188719640,13758,1188725640,6000,56,8
1523,10258535,20500178,2,180,2007-09-02 11:23:00,2007-09-02 12:14:00,13,13758,1188721380,13758,1188724440,3060,58,2
1126,10239189,20599380,1,180,2007-09-02 11:29:00,2007-09-02 12:19:00,13,13758,1188721740,13758,1188724740,3000,46,9
628,10125185,20546980,1,131,2007-09-02 11:32:00,2007-09-02 12:04:00,13,13758,1188721920,13758,1188723840,1920,22,10
1822,10272253,20542510,1,131,2007-09-02 11:33:00,2007-09-02 12:09:00,13,13758,1188721980,13758,1188724140,2160,19,5
1888,10274362,20546852,1,131,2007-09-02 11:33:00,2007-09-02 12:07:00,13,13758,1188721980,13758,1188724020,2040,23,6
585,10118419,20571383,1,110,2007-09-02 11:38:00,2007-09-02 12:56:00,13,13758,1188722280,13758,1188726960,4680,8,7
1823,10272254,20542511,1,131,2007-09-02 11:39:00,2007-09-02 12:02:00,13,13758,1188722340,13758,1188723720,1380,19,6
2243,10290854,20599848,1,180,2007-09-02 11:42:00,2007-09-02 14:17:00,13,13758,1188722520,13758,1188731820,9300,59,3
949,10193048,20605090,1,187,2007-09-02 11:46:00,2007-09-02 13:54:00,13,13758,1188722760,13758,1188730440,7680,19,3
